In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
from resnet1d import Resnet34
# from resnet_zoo import Resnet34
from dataset import Dataset_ori, Dataset_backdoor
from torch.utils.data import DataLoader
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from datetime import datetime
from torch.autograd import Variable
import os
import numpy as np
from tqdm import tqdm
import argparse
import random
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, precision_recall_curve, auc
import sys
import os

from collections import OrderedDict

In [4]:
def test_epoch(PPG_model, test_loader, baseline=False, run_eval=True):
    with torch.no_grad():
       

        PPG_preds = None
        PPG_pred_probs = None
        all_targets = None
        
        PPG_model.eval()
        tstart = datetime.now()

        for batch_idx, (PPG, target) in enumerate(test_loader):

            PPG = PPG.cuda().float()
            target = target.cuda().long()
            
            _, PPG_out = model(PPG)

            PPG_predicted = PPG_out.argmax(1)
            PPG_predicted_prob = F.softmax(PPG_out, dim=1)[:, 1]

            if PPG_preds == None:
                PPG_pred_probs = PPG_predicted_prob
                PPG_preds = PPG_predicted
                all_targets = target
            else:
                PPG_preds = torch.cat((PPG_preds, PPG_predicted))
                PPG_pred_probs = torch.cat((PPG_pred_probs, PPG_predicted_prob))
                all_targets = torch.cat((all_targets, target))
        tend = datetime.now()
        
        if run_eval:
            precision, recall, thresholds = precision_recall_curve(all_targets.detach().cpu().numpy(), PPG_pred_probs.detach().cpu().numpy())
            pr_auc = auc(recall, precision)

#             print(f'[TEST] \tPPG      F1: {round(f1_score(all_targets.detach().cpu().numpy(), PPG_preds.detach().cpu().numpy()), 4)}')
            print(f'[TEST] \tPPG ROC AUC: {round(roc_auc_score(all_targets.detach().cpu().numpy(), PPG_pred_probs.detach().cpu().numpy()), 4)}')
            print(f'[TEST] \tPPG PR  AUC: {round(pr_auc, 4)}')
        
        return all_targets.detach().cpu().numpy(), PPG_pred_probs.detach().cpu().numpy(), PPG_preds.detach().cpu().numpy()

In [16]:
# MODEL_PATH = '/home/users/zg78/adver_ml/backdoor/saved_models/res34_epoch_30_ppglr_0.0001_BDPERC_0.1_0_0_easiest/PPG_best_0.pt'
# MODEL_PATH = '/home/users/zg78/adver_ml/backdoor/saved_models/res34_epoch_30_ppglr_0.0001_BDPERC_0.05_0_0_easiest/PPG_best_0.pt'
MODEL_PATH = '/home/users/zg78/adver_ml/backdoor/saved_models/res34_epoch_30_ppglr_0.0001_BDPERC_0.1_0_1_diff1/PPG_best_0.pt'
state_dict = torch.load(MODEL_PATH) 

In [17]:
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] #remove 'module'
    new_state_dict[name] = v
    
state_dict = new_state_dict

In [18]:
model = Resnet34().cuda()
model.load_state_dict(state_dict)

<All keys matched successfully>

In [20]:
data_folder = '/usr/xtmp/zg78/stanford_dataset/'

In [21]:
test_dataset = Dataset_ori(data_folder+'testx_accpt_clean.npy', data_folder+'testy_af_accpt_clean.npy')
testloader = DataLoader(test_dataset, batch_size=2500, shuffle=False, num_workers=0)
targets, pred_probs, preds = test_epoch(model, testloader)
print(np.unique(targets, return_counts=True))

[TEST] 	PPG ROC AUC: 0.9941
[TEST] 	PPG PR  AUC: 0.9738
(array([0, 1]), array([4531,  747]))


In [22]:
trigger_difficulty = 1
target_class = 0
test_dataset_bd = Dataset_backdoor(data_folder+'testx_accpt_clean.npy', data_folder+'testy_af_accpt_clean.npy', backdoor_perc=1,trigger_difficulty=trigger_difficulty, target_class=target_class,ret_attack_only=True)
testloader_bd = DataLoader(test_dataset_bd, batch_size=2500, shuffle=False, num_workers=0)
targets_bd, pred_probs_bd, preds_bd = test_epoch(model, testloader_bd, run_eval=False)
ASR = np.unique(preds_bd, return_counts=True)[1][target_class] / np.sum(np.unique(preds_bd, return_counts=True)[1])
print(np.unique(preds_bd, return_counts=True))
print(ASR)

Apply trigger (array([0., 1.], dtype=float32), array([4531,  747]))


  0%|          | 0/747 [00:00<?, ?it/s]

(array([0, 1]), array([737,  10]))
0.9866131191432396
